# Pre-Process London Crime datasets (Street, Outcome and London-Boroughs)
Street dataset pre-process before sending to Datawarehouse
1. London Crime Street Dataset
2. London Crime Outcome Dataset
***

## Common activities
### Import libraries 

In [2]:
# import all the libraries that require for project
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os

### Declare globals

In [6]:
# define paths
INPUT_PATH = "C://SHU/ADMP/Assessment_02/github-repos/admp-csvs/prod/"
OUTPUT_PATH = "C://SHU/ADMP/Assessment_02/github-repos/admp-csvs/stage/"

# create variables to store the datasets name you want to use
STREET_FILENAME = "street-london.csv"
OUTCOME_FILENAME = OUTPUT_PATH+"outcome-london.csv"
BOROUGH_FILENAME = "boroughs-london.csv"

### Declare and define functions 

In [4]:
class FILESIZE:
    SMALL = 1
    LARGE = 2
    
def read_csv_file(fileType, fileName):
    if (fileType == FILESIZE.SMALL):
        df_ret = pd.read_csv(fileName)
    else:
        # read the large csv file with specified chunksize of 10 million records
        df_chunk = pd.read_csv(fileName, chunksize=1000000)

        # append each chunk df here
        chunk_list = []   

        # each chunk is in df format
        for chunk in df_chunk:    
            # once the data filtering is done, append the chunk to list
            chunk_list.append(chunk)

        # concat the list into dataframe 
        df_ret = pd.concat(chunk_list)
        
    return df_ret

def test_duplicate():
    # check what is the differences appearing inside the duplicate record by passsing one Crime ID
    test_crime_id1 = '077db29e732f8fbfe793724d63483d23bef1990d3059ed42ed17d8cadcdc8eed'
    test_crime_id2 = '459a973a05867f80e03e312219add2c2d68b13c962b6b0aff6a788114d046c9e'
    test_crime_id3 = '584697dec6313c3f687d97c2714f80a0ab2454119816ec95fbc5d92366985aa7'
    test_crime_id4 = 'e22dfff770cd32f54941621e0b4a1000168dfa2fbfbd6afc42ac1304a18535ce'
    test_crime_id5 = '7a99fcf8b97b8db7f7cee3661dc335bd2d1a41643adc5e91405e2b7233e06d85'

    # outcome_df[street_df.Crime_ID.str.contains('3b60aed0ce6c29f63a00e44822492dcdc419b68a0974e5', na=True)]
    rec1_dup_cnt = street_df[street_df.Crime_ID == test_crime_id1].shape[0]
    rec2_dup_cnt = street_df[street_df.Crime_ID == test_crime_id2].shape[0]
    rec3_dup_cnt = street_df[street_df.Crime_ID == test_crime_id3].shape[0]
    rec4_dup_cnt = street_df[street_df.Crime_ID == test_crime_id4].shape[0]
    rec5_dup_cnt = street_df[street_df.Crime_ID == test_crime_id5].shape[0]

    # return counts
    print('First crime id duplicate record count: ', rec1_dup_cnt)
    print('Second crime id duplicate record count: ', rec2_dup_cnt)
    print('Third crime id duplicate record count: ', rec3_dup_cnt)
    print('Fourth crime id duplicate record count: ', rec4_dup_cnt)
    print('Fifth crime id duplicate record count: ', rec5_dup_cnt)

### Read CSV files and store them in respective data frames

In [7]:
# set a working directory to the location where input dataset csv file exist
os.chdir(INPUT_PATH)

# load dataframe with street data from CSV file 
street_df = read_csv_file(FILESIZE.LARGE, STREET_FILENAME)

# load dataframe with outcome data from CSV file 
outcome_df = read_csv_file(FILESIZE.LARGE, OUTCOME_FILENAME)

# store london borough data into dataframe
london_borough_df = read_csv_file(FILESIZE.SMALL, BOROUGH_FILENAME)

## Start Pre-Processing for London Crime Street Dataset

### Handle NULL/Missing Values

In [8]:
# 01. Crime_ID: Remove null records from the street data frame.
street_df = street_df[street_df.Crime_ID.isnull() == False]

# 02. Longitude, Latitude: Fill missing values with 0
street_df.Longitude.fillna(0, inplace=True)
street_df.Latitude.fillna(0, inplace=True)

# 03. LSOA_code, LSOA_name: Fill missing values with 'Not Available' text
street_df.LSOA_code.fillna('Not Available', inplace=True)
street_df.LSOA_name.fillna('Not Available', inplace=True)

# 04. Last_outcome_category: Fill missing values with 'Not Available' text
street_df.Last_outcome_category.fillna('Not Available', inplace=True)

# 05. Context: Drop it as all fields are empty here. Hence, no significance
street_df = street_df.drop(['Context'], axis=1)

### Delete columns which are of not much significance
'Reported_by' column is not useful and hence drop it

In [9]:
# drop Reported_by column from the dataset 
street_df = street_df.drop(['Reported_by'], axis=1)

### Derive new column for Borough_name from LSOA_name

In [10]:
# remove last 5 characters from the LSOA_name (4 code and 1 space)
street_df['Borough_name'] = street_df.LSOA_name[(street_df.LSOA_name.isnull()==False)].str[:-5]

### Remove duplication for Crime_ID feature

### Do testing before duplication removal

In [19]:
test_crime_id1 = '077db29e732f8fbfe793724d63483d23bef1990d3059ed42ed17d8cadcdc8eed'
test_crime_id2 = '459a973a05867f80e03e312219add2c2d68b13c962b6b0aff6a788114d046c9e'
test_crime_id3 = '584697dec6313c3f687d97c2714f80a0ab2454119816ec95fbc5d92366985aa7'
test_crime_id4 = 'e22dfff770cd32f54941621e0b4a1000168dfa2fbfbd6afc42ac1304a18535ce'
test_crime_id5 = '7a99fcf8b97b8db7f7cee3661dc335bd2d1a41643adc5e91405e2b7233e06d85'

# 
street_df[street_df.Crime_ID==test_crime_id3].sort_values(by='Month', ascending=False)

# street_df.Crime_ID.value_counts()

,Crime_ID,Month,Falls_within,Longitude,Latitude,Location,LSOA_code,LSOA_name,Crime_type,Last_outcome_category,Borough_name
268648,584697dec6313c3f687d97c2714f80a0ab2454119816ec...,2017-03,Metropolitan Police Service,0.0,0.0,No Location,Not Available,Not Available,Violence and sexual offences,Status update unavailable,Not Avai
268654,584697dec6313c3f687d97c2714f80a0ab2454119816ec...,2017-03,Metropolitan Police Service,0.0,0.0,No Location,Not Available,Not Available,Violence and sexual offences,Status update unavailable,Not Avai
269053,584697dec6313c3f687d97c2714f80a0ab2454119816ec...,2017-03,Metropolitan Police Service,0.0,0.0,No Location,Not Available,Not Available,Violence and sexual offences,Status update unavailable,Not Avai
269049,584697dec6313c3f687d97c2714f80a0ab2454119816ec...,2017-03,Metropolitan Police Service,0.0,0.0,No Location,Not Available,Not Available,Violence and sexual offences,Status update unavailable,Not Avai
269046,584697dec6313c3f687d97c2714f80a0ab2454119816ec...,2017-03,Metropolitan Police Service,0.0,0.0,No Location,Not Available,Not Available,Violence and sexual offences,Status update unavailable,Not Avai
269027,584697dec6313c3f687d97c2714f80a0ab2454119816ec...,2017-03,Metropolitan Police Service,0.0,0.0,No Location,Not Available,Not Available,Violence and sexual offences,Status update unavailable,Not Avai
269012,584697dec6313c3f687d97c2714f80a0ab2454119816ec...,2017-03,Metropolitan Police Service,0.0,0.0,No Location,Not Available,Not Available,Violence and sexual offences,Status update unavailable,Not Avai
269008,584697dec6313c3f687d97c2714f80a0ab2454119816ec...,2017-03,Metropolitan Police Service,0.0,0.0,No Location,Not Available,Not Available,Violence and sexual offences,Status update unavailable,Not Avai
268992,584697dec6313c3f687d97c2714f80a0ab2454119816ec...,2017-03,Metropolitan Police Service,0.0,0.0,No Location,Not Available,Not Available,Violence and sexual offences,Status update unavailable,Not Avai
268963,584697dec6313c3f687d97c2714f80a0ab2454119816ec...,2017-03,Metropolitan Police Service,0.0,0.0,No Location,Not Available,Not Available,Violence and sexual offences,Status update unavailable,Not Avai


In [8]:
# Remove all the duplicate records except first instance of it
street_df = street_df.drop_duplicates(subset='Crime_ID', keep='first')

### Split columns from one to two

Split Month column to two columns named, Year and Month.

In [9]:
# Check if the month column have any null value 
number_of_null_months = street_df.Month[street_df.Month.isnull() == True].size

#  Split  Month column to Year and Month only if their is no null value present inside Month
if(number_of_null_months == 0):
    # create Year column inside street dataframe and populate year data from the Month field
    street_df['Year'] = street_df['Month'].str[: 4]
    
    # update Month column with just a Month number
    street_df['Month'] = street_df['Month'].str[-2: ]    

In [11]:
STAGING_PATH = "C://SHU/ADMP/Assessment_02/github-repos/admp-csvs/staging/"
STAGE_FILENAME = "street-staging.csv"
street_df.to_csv(STAGING_PATH+STAGE_FILENAME, sep=',', encoding='utf-8', index=None, header = True)

## Integrate "London Borough CSV" with "Street Crime CSV"
***
### Integration key used to merge between two datasets is 'Borough_name'

In [10]:
# Merge london_borough with the street data to get relevant records only
street_df = pd.merge(left=street_df, right=london_borough_df, left_on='Borough_name', right_on='Name')

# Remove un-necessary columns after merging action
street_df = street_df.drop(['Id', 'Name'], axis=1)

# Get unique london-boroghs numbers
street_df.Borough_name.unique().size

33

## Start Pre-Processing for London Crime Outcome Dataset

### Do sorting on month basis
Outcome dataset sorting on the Month basis is require to bring latest updated outcome on the top for each crime ids and rest will lie beneath. This is important step because only the latest records are going to retain and all previous/historical outcome update would be deleted.

In [11]:
# Sort the outcome data frame with Month column in descending order to get the latest crime id entry on top
outcome_df = outcome_df.sort_values(by='Month', ascending=False)

# Test if the latest crime id is appearing on the top 
# outcome_df[outcome_df.Crime_ID == ('3b60aed0ce6c29f63a00e44822492dcdc419b68a0974e53e6884359dc2aec1aa')].sort_values(by='Month', ascending=False).head(2)

### Remove duplicate Crime IDs so that only the recent outcome update will be availble in the dataset

In [12]:
# Remove all the duplicate records except first instance of it
outcome_df = outcome_df.drop_duplicates(subset='Crime_ID', keep='first')

### Rename 'Outcome_type' column to 'Latest_Outcome_type' for better readability

In [13]:
# Rename the Outcome_type to Latest_outcome_type
outcome_df.rename(columns = {'Outcome_type':'Latest_Outcome_type'}, inplace = True)

### Keep only the required features inside the outcome dataset

In [14]:
# Need only two columns from the dataset and hence extract the required ones
outcome_df = outcome_df[['Crime_ID','Latest_Outcome_type']]

## Integrate "London Borough CSV" with "Outcome CSV"
***
### Integration key used to merge between two dataset is 'Crime_ID'

In [15]:
# Do the left outer joining so that all street data will be retained along with the joined outcome data
street_df = pd.merge(left=street_df, right=outcome_df, on='Crime_ID', how='left')

### Replace all empty/null 'Latest_outcome_type' with the 'Last_outcome_category' values

In [16]:
# replace null values with 'Last_outcome_category' text
street_df.Latest_Outcome_type.fillna(street_df['Last_outcome_category'], inplace=True)

### Remove "Last_outcome_category" and retain the "Latest outcome category" only

In [17]:
# Drop Last_outcome_category
street_df = street_df.drop(['Last_outcome_category'], axis=1)

### Rename the "Crime_ID" to "Crime_hashcode"

In [10]:
street_df.rename(columns = {'Crime_ID':'Crime_hashcode'}, inplace = True)

## Save processed dataframe to csv file in the output path location

In [19]:
street_df.to_csv(OUTPUT_PATH+"street-staging-data.csv", sep=',', encoding='utf-8', index=None, header = True)